In [1]:
# Prerequisites:
#   1. Install required Python packages:
#      `pip install 'sagemaker[local]' 
#   2. Docker Desktop installed and running on your computer:
#      `docker ps`
#   3. You should have AWS credentials configured on your local machine
#      in order to be able to pull the docker image from ECR.

!pip install 'sagemaker[local]' --upgrade

%env AWS_ACCESS_KEY_ID=<Your Access Key>
%env AWS_SECRET_ACCESS_KEY=<Your Secret Key>
%env AWS_SESSION_TOKEN=<Your Session Token (if needed)>

#!aws configure set aws_access_key_id $AWS_ACCESS_KEY_ID
#!aws configure set aws_secret_access_key $AWS_SECRET_ACCESS_KEY
#!aws configure set aws_session_token $AWS_SESSION_TOKEN
#!aws configure set default.region $REGION

     |████████████████████████████████| 527 kB 270 kB/s eta 0:00:01
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached urllib3-1.26.8-py2.py3-none-any.whl (138 kB)
  Using cached PyYAML-5.4.1-cp39-cp39-macosx_10_9_x86_64.whl (259 kB)
  Using cached jsonschema-3.2.0-py2.py3-none-any.whl (56 kB)
  Created wheel for sagemaker: filename=sagemaker-2.94.0-py2.py3-none-any.whl size=740788 sha256=a13164e36129fe3f5ca9fe8349439cdec2900ca8bad568673c16688146119256
  Stored in directory: /Users/heichow/Library/Caches/pip/wheels/f3/9a/78/b8086493572f69114c0f44c9f08f301350360a931fea50e073
Successfully built sagemaker
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Uninstalling attrs-21.4.0:
      Successfully uninstalled attrs-21.4.0


  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.4.0
    Uninstalling jsonschema-4.4.0:
      Successfully uninstalled jsonschema-4.4.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.93.1
    Uninstalling sagemaker-2.93.1:
      Successfully uninstalled sagemaker-2.93.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
env: AWS_ACCESS_KEY_ID=<Your Access Key>
env: AWS_SECRET_ACCESS_KEY=<Your Secret Key>
e

In [2]:
import numpy as np
import pandas as pd
import os
from sklearn import datasets

# Load Iris dataset, then join labels and features together
iris = datasets.load_iris()
joined_iris = np.insert(iris.data, 0, iris.target, axis=1)

df = pd.DataFrame(joined_iris, columns = ['label','sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)'])
train_data, validation_data, test_data = np.split(df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))])

# Create a temporary directory and write the dataset as CSV
os.makedirs('./data', exist_ok=True)
#np.savetxt('./data/iris.csv', joined_iris, delimiter=',', fmt='%1.1f, %1.3f, %1.3f, %1.3f, %1.3f')
train_data.to_csv('data/train.csv', index=False, header=False)
validation_data.to_csv('data/validation.csv', index=False, header=False)
test_data.to_csv('data/test.csv', index=False, header=False)

In [3]:
# Split testing data and label for later model deploy test
test_y = test_data.iloc[:,0]
test_X = test_data.iloc[:,1:]

In [4]:
from sagemaker.local import LocalSession

sess_local = LocalSession()
sess_local.config = {'local': {'local_code': True} }

In [5]:
from sagemaker.sklearn.estimator import SKLearn

DUMMY_IAM_ROLE = 'arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001'

sklearn = SKLearn('train.py', #/opt/ml/code/<script file>
                  instance_type='local',
                  instance_count=1,
                  framework_version='0.20.0',
                  py_version='py3',
                  role=DUMMY_IAM_ROLE,
                  sagemaker_session=sess_local,
                  metric_definitions=[{"Name": "model_accuracy", 
                                       "Regex": "Model Accuracy: ([0-9.]+).*$"}
                                     ],
                  hyperparameters={'max_leaf_nodes': 30} 
                  #/opt/ml/input/config/hyperparameters.json
                )

sklearn.fit({"train": 'file://./data/train.csv', "test": 'file://./data/validation.csv'})

Using the short-lived AWS credentials found in session. They might expire while running.


Creating 5sy4ktpo5p-algo-1-c57e5 ... 
Creating 5sy4ktpo5p-algo-1-c57e5 ... done
Attaching to 5sy4ktpo5p-algo-1-c57e5
5sy4ktpo5p-algo-1-c57e5 | 2022-06-14 07:06:08,785 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
5sy4ktpo5p-algo-1-c57e5 | 2022-06-14 07:06:08,789 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
5sy4ktpo5p-algo-1-c57e5 | 2022-06-14 07:06:08,804 sagemaker_sklearn_container.training INFO     Invoking user training script.
5sy4ktpo5p-algo-1-c57e5 | 2022-06-14 07:06:08,809 sagemaker-training-toolkit INFO     Installing module with the following command:
5sy4ktpo5p-algo-1-c57e5 | /miniconda3/bin/python -m pip install . 
5sy4ktpo5p-algo-1-c57e5 | Processing /opt/ml/code
5sy4ktpo5p-algo-1-c57e5 |   DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this

5sy4ktpo5p-algo-1-c57e5 | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
5sy4ktpo5p-algo-1-c57e5 |   import imp
5sy4ktpo5p-algo-1-c57e5 | Training Started
5sy4ktpo5p-algo-1-c57e5 | Model Accuracy: 1.0
5sy4ktpo5p-algo-1-c57e5 | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
5sy4ktpo5p-algo-1-c57e5 |   pickler.file_handle.write(chunk.tostring('C'))
5sy4ktpo5p-algo-1-c57e5 | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
5sy4ktpo5p-algo-1-c57e5 |   pickler.file_handle.write(chunk.tostring('C'))
5sy4ktpo5p-algo-1-c57e5 | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/numpy_pickle

In [7]:
predictor = sklearn.deploy(initial_instance_count=1, instance_type='local')

print("Predictions: {}".format(predictor.predict(test_X.values)))
print("Actual: {}".format(test_y.values))

Using the short-lived AWS credentials found in session. They might expire while running.


!Predictions: [1. 0. 1. 1. 0. 1. 2. 2. 0. 1. 2. 2. 0. 2. 0. 1. 2. 2. 1. 2. 1. 1. 2. 2.
 0. 1. 2. 0. 1. 2.]
Actual: [1. 0. 1. 1. 0. 1. 2. 2. 0. 1. 2. 2. 0. 2. 0. 1. 2. 2. 1. 2. 1. 1. 2. 2.
 0. 1. 2. 0. 1. 2.]


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/heichow/opt/anaconda3/lib/python3.9/site-packages/sagemaker/local/image.py", line 852, in run
    _stream_output(self.process)
  File "/Users/heichow/opt/anaconda3/lib/python3.9/site-packages/sagemaker/local/image.py", line 914, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/heichow/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/heichow/opt/anaconda3/lib/python3.9/site-packages/sagemaker/local/image.py", line 857, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/private/var/folders/4w/06mq56fx0f5g0xq210xwp1rclh0tkw/T/tmpx6pw89g5/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited wit